In [32]:
from sklearn.metrics import f1_score
import pandas as pd
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm
import xgboost
import sklearn
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import class_weight

In [33]:
df_X = pd.read_csv("data/X_train_features_final.csv", index_col=0)
df_Y = pd.read_csv("data/y_train.csv", index_col="id")

In [50]:
from sklearn.model_selection import RandomizedSearchCV

random_grid = {'max_depth': [None],
        'min_samples_leaf': [1,2,3,4],
        'min_samples_split': [2,3,4,5],
        'n_estimators':[100,200,300, 400, 500]}
print(random_grid)

{'max_depth': [None], 'min_samples_leaf': [1, 2, 3, 4], 'min_samples_split': [2, 3, 4, 5], 'n_estimators': [100, 200, 300, 400, 500]}


In [51]:
et = ExtraTreesClassifier(class_weight = 'balanced')
et_random = RandomizedSearchCV(estimator = et, param_distributions = random_grid, scoring='f1_micro', n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1)

In [52]:
X = df_X.to_numpy()
Y = df_Y.to_numpy().squeeze()
et_random.fit(X, Y)

/Users/tdao/opt/anaconda3/envs/pyaml/lib/python3.10/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 80 is smaller than n_iter=100. Running 80 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 10 folds for each of 80 candidates, totalling 800 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.7s
[CV] END m

RandomizedSearchCV(cv=10,
                   estimator=ExtraTreesClassifier(class_weight='balanced'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': [None],
                                        'min_samples_leaf': [1, 2, 3, 4],
                                        'min_samples_split': [2, 3, 4, 5],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   random_state=42, scoring='f1_micro', verbose=2)

In [53]:
et_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_depth': None}

In [14]:
# best params
best_params = {'n_estimators': 400,
               'min_samples_split': 2,
               'min_samples_leaf': 1,
               'max_depth': None}

Evaluate

In [54]:
skf = StratifiedKFold(n_splits=5)
f1_base = []
f1_optim = []

for train_index, test_index in skf.split(X, Y):
    clf = ExtraTreesClassifier(class_weight = 'balanced')
    
    clf.fit(X[train_index], Y[train_index])
    y_pred = clf.predict(X[test_index])
    f1_base.append(f1_score(Y[test_index], y_pred, average='micro'))
    
    clf_optim = et_random.best_estimator_
    clf_optim.fit(X[train_index], Y[train_index])
    y_pred = clf_optim.predict(X[test_index])
    f1_optim.append(f1_score(Y[test_index], y_pred, average='micro'))

In [55]:
# knn
print(np.mean(f1_base))
print(np.mean(f1_optim))

0.8387707722385143
0.8415072473729228
